In [9]:
pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from Bio import SeqIO
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
neg_seq = []
neg_id= []

for seq_record in SeqIO.parse("/content/drive/MyDrive/Briefings in Bioinformatics/GKNEGTRAIN.txt", "fasta"):
    neg_seq.append(str(seq_record.seq))
    neg_id.append(str(seq_record.id))
pos_seq = []
pos_id= []

for seq_record in SeqIO.parse("/content/drive/MyDrive/Briefings in Bioinformatics/GKPOSTRAIN.txt", "fasta"):
    pos_seq.append(str(seq_record.seq))
    pos_id.append(str(seq_record.id))
seqs=pos_seq + neg_seq
dfclass = pd.DataFrame({'class' :  np.repeat((1,0), (206, 206))})
y=dfclass['class']
print(seqs)

['EIVPVTIPQRKGEPLVVEHDE', 'AIAVEAKDFPKTPGLVINELL', 'GTIEFLVDEQKNFYFLEMNTR', 'NDQTIPQLKAKVIAGSANNQL', 'KLIELDGTENKSKLGANAILG', 'IIEAIEKAGYKPGEQVMLAMD', 'HKLLTERLGKKVQLVGDDLFV', 'VGDDLFVTNTKKLAEGIEKGV', 'ISTGTLANMVKRGREALESNM', 'VEAHHGALSEKEAKRWERVYD', 'HRLETMTPLPKKALAFVRRLQ', 'SIVSTLTKHEKNVWDSLCLLL', 'RGKVKWFNNEKGYGFIEVEGG', 'OOOOMSVFVNKDTKVIVQGIT', 'LRDDVHVPTVKEAAKAGEEIE', 'LFSVYARCGAKTIEFGGWEMP', 'LKKNIGLALVKADVAAIGREV', 'QGVGFIYISHKLKEVQEIADR', 'PASRFHNAVEKNGAVHSOOOO', 'PVMEGKAMLFKEFAGVDAFPI', 'FIGVSAPGILKVEDVKKMARD', 'TVNEMIDAFTKARDDSNIGVI', 'NAVAETSGLSKKDATKAVDAV', 'OOOOOOOOMNKTELINAVAET', 'SITEALRKGDKVQLIGFGNFE', 'ASGETFETVIKDIVKGGLVAD', 'PEQGRLSLSIKEALPGPWEGL', 'GPWEGLGEKVKPGDVVTGTVK', 'RRIGSPHEVLKEGDVVKAKVL', 'ISELSNLHIEKPSDAVAVGDE', 'QLTIGQNAAAKNMEKLSSGLR', 'DYRPKYVAASKIKKQPGDYVV', 'SLPGIMKAKKKPLEELELDDL', 'TKTIEVFLPPKREAGKILQGE', 'AIEEAIQVRDKHGGEVTVVTV', 'KIAKDVAPKYKADLIVCGATG', 'QTIVVAVDGSKEAEWALKKAI', 'QYANELLERYKNEAIAAGLDD', 'VDIAVEFGSPKVKIAKDVAPK', 'IQQHLEAATDKEKEOOOOOOO',

In [13]:
Indneg_seq = []
Indneg_id= []

for Indseq_record in SeqIO.parse("/content/drive/MyDrive/Briefings in Bioinformatics/GKNEGIND.txt", "fasta"):
    Indneg_seq.append(str(Indseq_record.seq))
    Indneg_id.append(str(Indseq_record.id))
Indpos_seq = []
Indpos_id= []

for Indseq_record in SeqIO.parse("/content/drive/MyDrive/Briefings in Bioinformatics/GKPOSIND.txt", "fasta"):
    Indpos_seq.append(str(Indseq_record.seq))
    Indpos_id.append(str(Indseq_record.id))

Indseqs=Indpos_seq + Indneg_seq
Inddfclass = pd.DataFrame({'class' :  np.repeat((1,0), (17, 192))})
z=Inddfclass['class']
print(Indseqs)

['EPLTGLRVVGKFGDSVTTDHI', 'RLGGRLSELQKEGDISAKRGR', 'VTATNPDTGEKKEFEVIVRFD', 'PEMKDIDVPFKPSRVILQDFT', 'MSMYDACMKYKQDGTGLVVIA', 'NNGLFVAMPSKRTPDGEFRDI', 'VLLHSFMKEFKDVDQAITYVH', 'AAKGTFLLGIKTVIAESFERI', 'RFSALVVVGDKNGHVGFGTGK', 'FPGVSTGPEEKKMGIKSSSTR', 'RDIWPSMEEVKDVVKRAVDPE', 'RLMQYFLNMVKEAKMASSLKO', 'FSEKTCILPEKEVFOOOOOOO', 'KRRIEAGDIGKPIYFKGITRD', 'GGGVIIPREIKELHEYGIARI', 'DLLSEPIGKGKDGSDVYFRDI', 'OOOOMIVVEGKDDTAAIRRAV', 'OOOMRRIDPNKLELEERVVAV', 'ERVVAVNRVAKVVKGGRRLRF', 'VAVNRVAKVVKGGRRLRFSAL', 'KNGHVGFGTGKAQEVPEAIRK', 'KAQEVPEAIRKAIEDAKKNLI', 'EAIRKAIEDAKKNLIEVPIVG', 'GHFGAGEIILKPASEGTGVIA', 'ELAGISDILSKSIGSNTPINM', 'NMVRATFDGLKQLKRAEDVAR', 'RAEDVARLRGKTVEELLGOOO', 'LRRVNTEGRMKAIASITLDNE', 'AHPINSATRGKIQEAILAEYH', 'AILAEYHRLGKLEEELEEAGA', 'OOMSRLPYSIKVLLESVLRQV', 'NLAKWGTPEMKDIDVPFKPSR', 'PVVVDLASMRKAMADLGGDPY', 'ALEYNMDLEFKRNAERYKFLK', 'LEFKRNAERYKFLKWAQKAFD', 'PEVIGVRLTGKLPDGATATDL', 'LALKVTQVLRKKGVVGKFVEF', 'QVLRKKGVVGKFVEFFGPGVA', 'HHVQVVEAYCKANGLFYTPDA', 'SEIETNLSGPKRPQDLIPLSK',

In [14]:
pip install keras-tuner --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 27.2 MB/s 


In [15]:
import keras_tuner as kt

In [16]:
max_words = 5000
max_len = 350
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(seqs)
sequences = tok.texts_to_sequences(seqs)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [17]:
#Independent Testing
indtok = Tokenizer(num_words=max_words)
indtok.fit_on_texts(Indseqs)
Indsequences = indtok.texts_to_sequences(Indseqs)
Indsequences_matrix = sequence.pad_sequences(Indsequences,maxlen=max_len)
X_train=sequences_matrix
y_train=y
X_test=Indsequences_matrix
y_test=z

In [18]:
import keras_tuner as kt
from tensorflow import keras

In [19]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(hp.Choice('units', [2, 4, 8, 16, 32, 64, 128]),activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',metrics=['accuracy']) 
  return model

In [20]:
tuner1 = kt.RandomSearch(
    build_model,
    overwrite=True,
    objective='val_accuracy',
    max_trials=10,
    directory='./multiclass_classifier/training')

In [21]:
tuner1.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))
best_model = tuner1.get_best_models()[0]

Trial 7 Complete [00h 00m 03s]
val_accuracy: 0.9904305934906006

Best val_accuracy So Far: 0.9952152967453003
Total elapsed time: 00h 00m 31s
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner1.results_summary()

Results summary
Results in ./multiclass_classifier/training/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 64
Score: 0.9952152967453003
Trial summary
Hyperparameters:
units: 32
Score: 0.9952152967453003
Trial summary
Hyperparameters:
units: 8
Score: 0.9904305934906006
Trial summary
Hyperparameters:
units: 16
Score: 0.9904305934906006
Trial summary
Hyperparameters:
units: 2
Score: 0.9856459498405457
Trial summary
Hyperparameters:
units: 128
Score: 0.9665071964263916
Trial summary
Hyperparameters:
units: 4
Score: 0.9186602830886841


In [23]:
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (412, 64)                 22464     
                                                                 
 dense_1 (Dense)             (412, 1)                  65        
                                                                 
Total params: 22,529
Trainable params: 22,529
Non-trainable params: 0
_________________________________________________________________


In [24]:
accr = best_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7/7 [==============================] - 0s 3ms/step - loss: 0.4097 - accuracy: 0.9952
Test set
  Loss: 0.410
  Accuracy: 0.995


In [25]:
pred=best_model.predict(X_test)
pred1 = np.round_(pred)

from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(y_test, pred1)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(y_test, pred1)

MCC:  0.9676259135453443


array([[192,   0],
       [  1,  16]])

In [26]:
from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(y_test, pred1)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(y_test, pred1)
print(classification_report(y_test, pred1))
tp, fn, fp, tn = confusion_matrix(y_test, pred1).ravel()
print("MCC ---> {0}".format(mc))
print("Confusion Matrix. tn, fp, fn, tp ---> ", tp, fn, fp, tn)
print("Precision --->TP/TP+Fp ", tp/(tp+fp))
print("Recall - SN - Sensitivity --->TP/TP+FN ", tp/(tp+fn))
print("Specificity - SP ---> ", tn/(tn+fp))
print("Balanced Accuracy ---> ", ((tp/(tp+fn))+(tn/(tn+fp)))/2)
print("Jaccard Index --->TP/TP+FN+FP ", tp/(tp+fn+fp))

MCC:  0.9676259135453443
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       192
           1       1.00      0.94      0.97        17

    accuracy                           1.00       209
   macro avg       1.00      0.97      0.98       209
weighted avg       1.00      1.00      1.00       209

MCC ---> 0.9676259135453443
Confusion Matrix. tn, fp, fn, tp --->  192 0 1 16
Precision --->TP/TP+Fp  0.9948186528497409
Recall - SN - Sensitivity --->TP/TP+FN  1.0
Specificity - SP --->  0.9411764705882353
Balanced Accuracy --->  0.9705882352941176
Jaccard Index --->TP/TP+FN+FP  0.9948186528497409
